In [1]:
!pip install transformers datasets accelerate nvidia-ml-py3

Using legacy 'setup.py install' for nvidia-ml-py3, since package 'wheel' is not installed.
    Running setup.py install for nvidia-ml-py3: started
    Running setup.py install for nvidia-ml-py3: finished with status 'done'


You should consider upgrading via the 'c:\users\yaroslav\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [1]:
import numpy as np
from datasets import Dataset


seq_len, dataset_size = 512, 512
dummy_data = {
    "input_ids": np.random.randint(100, 30000, (dataset_size, seq_len)),
    "labels": np.random.randint(0, 1, (dataset_size)),
}
ds = Dataset.from_dict(dummy_data)
ds.set_format("pt")

In [2]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [3]:
print_gpu_utilization()

GPU memory occupied: 753 MB.


In [6]:
!nvidia-smi

Sun Apr 16 15:21:15 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.14                 Driver Version: 531.14       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 L...  WDDM | 00000000:01:00.0  On |                  N/A |
| N/A   64C    P8               14W /  N/A|    618MiB /  6144MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [13]:
import pandas as pd

from googletrans import Translator


COVID_DATA_PATH = "data/preprocessed/world_news_test"

data = pd.read_json(COVID_DATA_PATH, orient="records", lines=True)

def translate_text(row: pd.DataFrame) -> str:
    translator = Translator()
    return translator.translate(row['text'], src='en', dest='pl').text

def translate_dataset(dataset) -> None:
    dataset['text'] = dataset.apply(translate_text, axis=1)
    return dataset

translate_dataset(data)

,text,label,hashtags,emojis,polarity,subjectivity,sentiment
0,Dom Aide Dint Homeysaid najgorsza osoba Turn C...,1,[],[],-0.009826,0.482797,negative
1,Flynn Hillary Clinton Big Woman Campus Embuse ...,0,[],[],0.098977,0.487655,positive
